**This is a problem where you will be given images of plant seedling belonging to 12 different classes and your model should be able to predict to which class the test images belong to. You can get this dataset from [here](http://www.kaggle.com/competitions/plant-seedlings-classification)**

**I'm using TRANSFER LEARNING technique to tackle this problem. To achive this, we will be importing and loading a pre-trained model(about which you can refer [here](http://keras.io/api/applications/inceptionresnetv2/) and it's weights and then we remove the top layer of the pre-trained model and then add some fully connected layers and output layers according to our requirment.**

**Now let us start by importing the required libraries and model. I'm using [InceptionResNetV2](http://keras.io/api/applications/inceptionresnetv2/) model. You are free to choose any pre-trained model.**

In [1]:
import pandas as pd

In [2]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# I'm importing preprocess_input function to preprocess our image data 
# before passing it to the model for it's better understanding and performance.
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [3]:
# Now I'm importing ImageDataGenerator to read the images and for data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Now I'm specifying the paths to the directories where our train and test images are present
train_path = '../input/seedlings/plant-seedlings-classification/train'
test_path = '../input/seedlings/plant-seedlings-classification/test'

In [5]:
# I'm using a keras functional API model in this problem as it is more flexible compared to
# keras sequential model
from tensorflow.keras.models import Model
# Now I'm importing the required layes
from tensorflow.keras.layers import Dense,Flatten

In [6]:
# initializing our model with the model's base as pre-trained model
# Giving the include_top parameter a false to remove the top layer of the pre-trained model
# weights = 'imagenet' stands for keeping the original weights of the model
base_model = InceptionResNetV2(include_top=False,input_shape=(299,299,3),weights='imagenet')

2022-09-29 05:26:21.568415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 05:26:21.692078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 05:26:21.692904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 05:26:21.694616: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

219070464/219055592 [==============================] - 1s 0us/step


In [7]:
# This step makes sure that the base_model does not update its weights during training.
for layer in base_model.layers:
    layer.trainable=False

**Now we will be building a model of our own with few layers on the top of the pre-trained model**

**Now, I will be passing the output we recieve by training our data on the pre trained model to the next layers which we built. The key advantage of this is that we train our data on the base_model only once and we pass that output as input to the layers which we built on the base_model, rather than once per epoch. It is a lot more cheaper and faster**

**But remember - By doing this,  it doesn't allow you to dynamically modify the input data of your new model during training, which is required when doing data augmentation. So for very small datasets, where data augmentation is very important, this approach doesn't go well**

In [8]:
# Now I'm passing the output of the base_model as input to flatten layer, which falttens out 
# the dimensions of tensord to 1-D array.
flatten_layer = Flatten()(base_model.output)

In [9]:
# Now I'm adding fully-connected layers(Dense Layers)
dense_layer_1 = Dense(256,activation='relu')(flatten_layer)
dense_layer_2 = Dense(128,activation='relu')(dense_layer_1)

In [10]:
# Now adding an output layer with 12 neurons as our data has images belonging to 12 classes
# This layer predicts to which class the image belongs to
prediction_layer = Dense(12,activation='softmax')(dense_layer_2)

**You can learn more about activation functions[ here](http://keras.io/api/layers/activations/)**

In [11]:
# Now I'm creating the final model and compiling it to train our data.
model = Model(inputs=base_model.input,outputs=prediction_layer)

**To compile our model, I'm using adam optimizer and loss as categorical_crossentropy which works well with multi-class image classification. You can learn more about them [here](http://keras.io/api/optimizers/)**

In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
# I'm not passing many command inside of ImageDataGenerator 
# This is because, as I mentioned, this method doesn't effectively augment our data
train_img_gen = ImageDataGenerator(rescale=1./255,horizontal_flip=True)

In [14]:
# reading the input using flow_from_directory method with a batch size of 32 images per batch
# I have passed target size as (299,299) as InceptionResNetV2 works well with (299,299,3)
train_images = train_img_gen.flow_from_directory(train_path,batch_size=32,
                                                target_size=(299,299),class_mode='categorical')

Found 4750 images belonging to 12 classes.


In [15]:
# Now It is time to train our model.
model.fit(train_images,epochs=10,steps_per_epoch=len(train_images))

2022-09-29 05:27:04.789905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-09-29 05:27:16.855320: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


149/149 [==============================] - 140s 823ms/step - loss: 3.4385 - accuracy: 0.5295
Epoch 2/10
149/149 [==============================] - 68s 452ms/step - loss: 0.9229 - accuracy: 0.7556
Epoch 3/10
149/149 [==============================] - 68s 455ms/step - loss: 0.4880 - accuracy: 0.8522
Epoch 4/10
149/149 [==============================] - 68s 459ms/step - loss: 0.3082 - accuracy: 0.9034
Epoch 5/10
149/149 [==============================] - 67s 451ms/step - loss: 0.2249 - accuracy: 0.9269
Epoch 6/10
149/149 [==============================] - 67s 447ms/step - loss: 0.1801 - accuracy: 0.9402
Epoch 7/10
149/149 [==============================] - 66s 446ms/step - loss: 0.1419 - accuracy: 0.9522
Epoch 8/10
149/149 [==============================] - 67s 452ms/step - loss: 0.1557 - accuracy: 0.9488
Epoch 9/10
149/149 [==============================] - 68s 454ms/step - loss: 0.1474 - accuracy: 0.9512
Epoch 10/10
149/149 [==============================] - 69s 462ms/step - loss: 0.128

**Now I'm reading in the test images to run our model and predict to which class they belong to**

In [16]:
test_img_gen = ImageDataGenerator(rescale=1./255)

In [18]:
test_images = test_img_gen.flow_from_directory(test_path,batch_size=1,
                                              target_size=(299,299),class_mode='categorical',
                                              shuffle=False)

Found 794 images belonging to 1 classes.


In [19]:
# this command returns the probabilities of an image belonging to each calss
predictions = model.predict(test_images)

In [20]:
# We have 794 images belonging to 12 classes in test directory.
# So shape of our prediction array should be (794,12)
predictions.shape

(794, 12)

In [21]:
predictions

array([[1.45693952e-17, 5.04031981e-08, 5.97395633e-10, ...,
        4.04844615e-14, 1.00000000e+00, 4.19060245e-18],
       [2.82539446e-07, 7.56660630e-13, 1.01753085e-11, ...,
        1.37918567e-13, 2.80611987e-14, 6.99295811e-07],
       [2.08533146e-09, 6.45526063e-11, 1.68918712e-06, ...,
        1.52710852e-12, 8.46909365e-15, 9.98142481e-01],
       ...,
       [7.74813325e-10, 9.04298247e-10, 2.43781342e-05, ...,
        5.11886866e-10, 7.03230216e-15, 9.99948740e-01],
       [4.22043356e-15, 9.84586835e-01, 1.54098514e-02, ...,
        3.24243319e-06, 9.58365760e-08, 2.22792162e-12],
       [1.26538402e-09, 6.08205084e-11, 8.36964553e-10, ...,
        6.68809539e-16, 1.07062970e-11, 9.29133073e-12]], dtype=float32)

**Now in the following steps, I'm converting our predictions into a dataframe, where ecah image is assigned with a class to which it belongs to. This is done by assigning the class with highest probability to our image**

In [22]:
species = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", 
           "Fat Hen","Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse",
           "Small-flowered Cranesbill","Sugar beet"]

classes = []

In [23]:
# identifying the highest probability and assiging it as the class to our image
for i in range(0, 794):
    x = predictions[i, :].argmax(axis=0)
    classes += [species[x]]

In [24]:
final_df = pd.DataFrame()
final_df['file'] = test_images.filenames
final_df['file'] = final_df['file'].str.replace('test/', '')
final_df['species'] = classes

In [25]:
# you can have a smaple look at first 50 predictions!!!
final_df.head(50)

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet
5,00d090cde.png,Loose Silky-bent
6,00ef713a8.png,Common Chickweed
7,01291174f.png,Fat Hen
8,026716f9b.png,Loose Silky-bent
9,02cfeb38d.png,Black-grass


**Whooo!!! Now, we have successfully built a model to achieve multi-class image classification**